## Note
### 1. 线性回归模型
- x：特征矩阵
- w：权值向量，同线性模型的参数向量
- b：截距
$$y=x\cdot w+b$$

### 2. 二分类逻辑回归模型
- Y：标签
- 左侧为Y=1的条件概率
$$P(Y=1|x)=\frac{1}{1+e^{-(x\cdot w+b)}}$$

### 3. 极大似然与损失函数
- 设：
$$P(Y=1|x)=p(x)$$
$$P(Y=0|x)=1-p(x)$$
- 通过极大似然使得概率值逼近标签值
    - xi:一个样本
    - yi：i样本对应的真实标签
$$MLE=ArgMax \prod [p(x_{i})]^{y_{i}}*[1-p(x_{i})]^{1-y_{i}}$$
- 损失函数 logloss
    - n：样本数
$$LogLoss = ArgMin-\frac{1}{n}(\sum_{i=1}^{n}(y_{i}*ln(p(x_{i}))+(1-y_{i})*ln(1-p(x_{i})))$$

### 4. 梯度下降
$$g_{i} = \frac{\partial LogLoss}{\partial w}=\frac{1}{n}\sum_{i=1}^{n}x_{i}*(p(x_{i})-y_{i})$$

- 通过迭代，更新w得到最优损失函数
    - j：第j轮迭代
    - lambda：学习率
$$w_{i}^{j+1}=w_{i}^{j}-\lambda g_{i}$$

### 5. 逻辑回归最终预测模型
$$LogisticModel=\left \{ 
\begin{aligned}
1 \quad \frac{1}{1+e^{-(x\cdot w+b)}}>0.5 \\
-1  \quad \frac{1}{1+e^{-(x\cdot w+b)}}\leq 0.5
\end{aligned}
\right.$$

In [23]:
import numpy as np # version: 1.20.1
import matplotlib.pyplot as plt # version: 3.3.4
from sklearn.datasets import load_breast_cancer # sklearn version: 0.24.1
from sklearn.preprocessing import StandardScaler

In [27]:
class logistic_regression():
    def __init__(self, learning_rate=0.01, max_iter=1000, batch_size=1):
        self.learning_rate = learning_rate
        self.max_iter = max_iter # maximum number of iterations
        self.batch_size = batch_size # batch size of the gradient descent. If 1, it is SGD
        self.w_history = [] # store weights at each iteration during the training
        self.cost_history = [] # store cost at each iteration during the training
    
    def fit(self, xtrain, ytrain):
        # train the model with the input training data, accept ndarray
        xtrain = self.augmentation(xtrain) # add bias column to X
        
        # batch gradient descent
        self.w_history, self.cost_history = self.gredient_descent(xtrain, ytrain)
        
        return self
    
    def gredient_descent(self, x, t):
        # x is training data, t is target
        n, m = x.shape
        w = np.zeros(m) # initialize w
        
        w_history = [] # store history weights in the processing of training
        cost_history = []
        for i in range(self.max_iter):

            sample_index = np.random.randint(0, n, self.batch_size) # stochastic selection of samples
            xx = x[sample_index]
            tt = t[sample_index]

            z = np.dot(w, xx.T) # calculate X*w, shape=(n,1)
            y = self.sigmoid(z) # calculate sigmoid(X*w), shape=(n,1)

            gd = 1/n * np.dot(xx.T, (y - tt)) # calculate gredient

            w = w - self.learning_rate * gd # update w

            w_history.append(w) # record each w

            self.w = w # update w
            
            # based on the updated weights compute cost for the training data: log loss or cross entropy loss
            z = np.dot(self.w, x.T)
            y = self.sigmoid(z)
            train_loss = self.cross_entropy_loss(t, y)
            train_cost = train_loss.sum() / len(y)
            
            cost_history.append(train_cost)
        
        return w_history, cost_history
    
    def predict(self, x, threshold=0.5, add_bias=True, return_proba=False):
        # convert to ndarray
        if not isinstance(x, np.ndarray):
            x = x.values
        # add bias column to x
        if add_bias:
            x = self.augmentation(x)
        z = np.dot(x, self.w)
        yhat = self.sigmoid(z).ravel()
        if return_proba:
            return yhat
        else:
            return (yhat >= threshold) * 1
        
    def augmentation(self, x):
        # add bias column to x
        n, m = x.shape
        ones = np.ones((n, 1))
        return np.hstack((ones, x))
    
    def sigmoid(self, x):
        # sigmoid function
        return 1 / (1+np.exp(-x))
    
    def cross_entropy_loss(self, y, yhat):
        # calculate cross entropy loss : -y*log(yhat) - (1-y)*log(1-yhat)

        loss = -y*np.log(yhat) - (1-y)*np.log(1-yhat)
        return loss

def split_data(data, target, ratio=0.8):
    n = data.shape[0]
    ind = np.arange(n)
    split = int(n*ratio)
    return data[:split], target[:split], data[split:], target[split:]

def confusion_matrix(y, y_pred):
    # binary classification confusion matrix
    ind_t = np.nonzero(y==1)[0] # positive label index
    ind_f = np.nonzero(y==0)[0] # negative label index
    tp = (y_pred[ind_t]==1).sum() # true positive
    fn = (y_pred[ind_t]==0).sum() # false negative
    tn = (y_pred[ind_f]==0).sum() # true negative
    fp = (y_pred[ind_f]==1).sum() # false negative
    return tp,fn,tn,fp

def misclassification_rate(y, y_pred):
    return (y!=y_pred).sum() / len(y)

def precision(y, y_pred):
    # precision = TP / (TP+FP)  
    tp,fn,tn,fp = confusion_matrix(y, y_pred)
    return tp / (tp+fp)

def recall(y, y_pred):
    # recall = TP / (TP+FN)
    tp,fn,tn,fp = confusion_matrix(y, y_pred)
    return tp / (tp+fn)

def f1_score(y, y_pred):
    # calculate f1 score = 2* (precision*recall) / (precision+recall)
    p = precision(y, y_pred)
    r = recall(y, y_pred)
    return 2 * (p*r) / (p+r)

In [28]:
# load data
iris = load_breast_cancer()
data = iris.data # (569, 30)
target = iris.target # (569,), label:0,1

# split data
xtrain, ytrain, xtest, ytest = split_data(data, target, 0.8)

# standardization
std = StandardScaler().fit(xtrain)
std_xtrain = std.transform(xtrain)
std_xtest = std.transform(xtest)

In [34]:
# train model
batch_size = len(std_xtrain) # using full batch
my_lr = logistic_regression(learning_rate=0.01, max_iter=100, batch_size=batch_size)
my_lr = my_lr.fit(std_xtrain, ytrain)

# predict testing set using different metrics
yhat = my_lr.predict(std_xtest)
print('f1 score of testing set =', f1_score(ytest, yhat))
print('misclassification rate =', misclassification_rate(ytest, yhat))

f1 score of testing set = 0.9714285714285714
misclassification rate = 0.043859649122807015
